In [11]:
'''
11/27/2023: outdated in fovor of notebook of the same name.
This script analyzes the results of a set of experiments conducted on different repositories.
It reads the experiment results from various directories and generates a master ledger containing the results.

The script performs the following steps:
1. Imports necessary libraries and modules.
2. Defines the paths to the result repositories.
3. Initializes variables and data structures.
4. Checks if a master ledger file exists. If it does, it loads the existing ledger. Otherwise, it creates a new one.
5. Iterates over each repository, fold, tune, model, and iteration to read the experiment results.
6. Processes the experiment results and updates the master ledger.
7. Saves the master ledger to a pickle file.
8. Defines a function to process each ledger entry in parallel.
9. Executes the processing function using a process pool executor.
10. Collects the results from the processing function.
11. Calculates the execution time and logs the results.
12. Concatenates the ledger results into a single dataframe.
13. Saves the master ledger dataframe to a pickle file.
'''
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import classification_report, confusion_matrix
import warnings
import multiprocessing as mp
from multiprocessing import Pool
import time
import logging
from tqdm import tqdm
warnings.filterwarnings("ignore")
logging.basicConfig(filename='logs/analyze_all_case_results.log',filemode='w', level=logging.INFO)

all_results_repos = {'name': 'Path'}
all_results_repos['anxietyE2E/results_archive'] = '/mnt/d/Users/alkurdi/anxietyE2E/results_archive'
all_results_repos['anxietyE2E/results'] = '/mnt/d/Users/alkurdi/anxietyE2E/results'
all_results_repos['anxietyE2E/results_reduced'] = '/mnt/d/Users/alkurdi/anxietyE2E/results_reduced'
all_results_repos['wsl2 may2023'] = '/mnt/d/Users/alkurdi/from_wsl2_created_may_23/wsl2_e2e/results'
all_results_repos['delta old e2e_snr_0.5'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e/results_snr_0.5'
all_results_repos['delta old e2e_copy_snr_0.1'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy/results_snr_0.1'
all_results_repos['delta old e2e_copy_snr_0.2'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy/results_snr_0.2'
all_results_repos['delta old e2e_copy_snr_0.5'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy/results_snr_0.5'
all_results_repos['delta old e2e_copy2_snr_0.01'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy2/results_snr_0.01'
all_results_repos['delta old e2e_copy2_snr_0.1'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy2/results_snr_0.1'
all_results_repos['delta old e2e_copy2_snr_0.2'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy2/results_snr_0.2'
all_results_repos['delta old e2e_copy2_snr_0.5'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy2/results_snr_0.5'
all_results_repos['delta old e2e_snr_0.2_snr_0.2'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_snr_0.2/results_snr_0.2'
all_results_repos['delta old e2e_snr_0.2_snr_0.5'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_snr_0.2/results_snr_0.5'
all_results_repos['bbyn katerina snr_0.5'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e_bbyn/katerina/results_snr_0.5'
all_results_repos['bbyn katerina snr_0.5_save'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e_bbyn/katerina/results_snr_0.5_save'

all_results_repos['new_delta_results/results_reduced'] = '/mnt/d/Users/alkurdi/new_delta_results/results_reduced'
all_results_repos['new_delta_results/results_reduced_before_completion'] = '/mnt/d/Users/alkurdi/new_delta_results/results_reduced_before_completion'
all_results_repos['new_delta_results/binary_results'] = '/mnt/d/Users/alkurdi/new_delta_results/binary_results_home'
all_results_repos['new_delta_results/results_baseline_3class'] = '/mnt/d/Users/alkurdi/new_delta_results/results_baseline_3class_home'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
done_list = {}
master_ledger = {'name':{'df_best_model':{}, 'df_metrics':{}, 'history':{}, 'result':{}, 'aggregated_classification_reports':{}, 'done':{}}}
i=0
force_update = False # range once on. 11/28/2023
if os.path.isfile('/mnt/d/Users/alkurdi/master_ledger.pkl') and force_update != True:
    with open('/mnt/d/Users/alkurdi/master_ledger.pkl', 'rb') as handle:
        master_ledger = pickle.load(handle)
    print('loaded master ledger')
else:
    for name, Path in all_results_repos.items():
        if name != 'name':        
            fold_gen = (i for i in os.listdir(Path) if 'WESAD' in i)
            for fold in fold_gen:
                tune_gen = (i for i in os.listdir(Path+'/'+fold) if 'tune' in i)
                for tune in tune_gen:
                    model_gen = (i for i in os.listdir(Path+'/'+fold+'/'+str(tune)))
                    for model in model_gen:
                        it_gen = ( i for i in os.listdir(Path+'/'+fold+'/'+str(tune)+'/'+model) if 'it' in i)
                        for it in it_gen:
                            i+=1
                            inst_name = str(name+'_'+fold+'_'+tune+'_'+model+'_'+it)
                            master_ledger[inst_name] = {}
                            try: 
                                in_path = Path+'/'+fold+'/'+str(tune)+'/'+model+'/'+str(it)
                                df_best_model = pd.read_csv(in_path+'/df_best_model.csv')
                                df_metrics = pd.read_csv(in_path+'/df_metrics.csv')
                                history = pd.read_csv(in_path+'/history.csv')
                                aggregated_classification_reports = {}
                                #aggregated_confusion_matrices = {}
                                metrics = ["precision", "recall", "f1-score", "support"]
                                with open(in_path+'/predictions.txt') as f:
                                    y_true = [int(x) for x in f.readline().split()]
                                    y_pred = [int(x) for x in f.readline().split()]
                                if len(aggregated_classification_reports) == 0:
                                    for clas in set(y_true):
                                        aggregated_classification_reports[clas] = {"f1-score": [], "precision": [], "recall": [],
                                                                                    "support": []}
                                report = classification_report(y_true, y_pred, output_dict=True)
                                cm = confusion_matrix(y_true, y_pred)
                                for clas in aggregated_classification_reports:
                                    for metric in metrics:
                                        aggregated_classification_reports[clas][metric].append(report[str(clas)][metric]) # type: ignore
                                aggregated_confusion_matrices = cm
                                result = []
                                for clas in aggregated_classification_reports:
                                    row = ['WESAD', clas]
                                    for metric in metrics:
                                        metric_values = aggregated_classification_reports[clas][metric]
                                        row.append(np.mean(metric_values))
                                        row.append(np.std(metric_values))
                                    result.append(row[:-1])  # Remove support std
                                is_done = os.path.isdir(in_path+'/DONE')
                                if is_done:
                                    done_list[in_path] = is_done            
                                master_ledger[inst_name]['df_best_model'] = df_best_model # type: ignore
                                master_ledger[inst_name]['df_metrics'] = df_metrics # type: ignore
                                master_ledger[inst_name]['history'] = history # type: ignore
                                master_ledger[inst_name]['result'] = result # type: ignore
                                master_ledger[inst_name]['aggregated_classification_reports'] = aggregated_classification_reports
                                master_ledger[inst_name]['aggregated_confusion_matrices'] = aggregated_confusion_matrices # type: ignore
                                master_ledger[inst_name]['is_done'] = is_done           # type: ignore
                            except Exception as e:
                                master_ledger[inst_name]['df_best_model'] = None # type: ignore
                                master_ledger[inst_name]['df_metrics'] = None # type: ignore
                                master_ledger[inst_name]['history'] = None # type: ignore
                                master_ledger[inst_name]['result'] = None # type: ignore
                                master_ledger[inst_name]['aggregated_classification_reports'] = None # type: ignore
                                master_ledger[inst_name]['aggregated_confusion_matrices'] = None # type: ignore
                                master_ledger[inst_name]['is_done'] = None # type: ignore
                                print('during '+name+'_'+fold+'_'+tune+'_'+model+'_'+it+' an error occured: ')
                                print(e)
    print('done. Processed '+str(i)+' instances. Thank you for your cooperation')                                               
    with open('/mnt/d/Users/alkurdi/master_ledger.pkl', 'wb') as handle:
        pickle.dump(master_ledger, handle, protocol=pickle.HIGHEST_PROTOCOL)

loaded master ledger


In [3]:
master_ledger['anxietyE2E/results_archive_WESAD_5fold_00_tune_00_cnnM_it_00']['aggregated_classification_reports']


{0: {'f1-score': [0.837037037037037],
  'precision': [0.738562091503268],
  'recall': [0.9658119658119658],
  'support': [117]},
 1: {'f1-score': [0.8403361344537816],
  'precision': [0.9259259259259259],
  'recall': [0.7692307692307693],
  'support': [65]},
 2: {'f1-score': [0.11764705882352941],
  'precision': [0.23076923076923078],
  'recall': [0.07894736842105263],
  'support': [38]}}

In [4]:
master_ledger['anxietyE2E/results_archive_WESAD_5fold_00_tune_00_cnnM_it_00']['aggregated_confusion_matrices']
cm = master_ledger['anxietyE2E/results_archive_WESAD_5fold_00_tune_00_cnnM_it_00']['aggregated_confusion_matrices']
print(cm)

[[113   4   0]
 [  5  50  10]
 [ 35   0   3]]


In [5]:
cm.ravel()

array([113,   4,   0,   5,  50,  10,  35,   0,   3])

In [6]:
master_ledger['anxietyE2E/results_archive_WESAD_5fold_00_tune_00_cnnM_it_00']['aggregated_confusion_matrices'].shape == (3,3)

True

In [7]:
master_ledger['anxietyE2E/results_archive_WESAD_5fold_00_tune_00_cnnM_it_00']['aggregated_confusion_matrices']

array([[113,   4,   0],
       [  5,  50,  10],
       [ 35,   0,   3]])

In [8]:
(cm.ravel())

array([113,   4,   0,   5,  50,  10,  35,   0,   3])

In [9]:
TP = cm.ravel()[4]
FN = cm.ravel()[3:6:2].sum()
FP = cm.ravel()[1:8:6].sum() # type: ignore
print('tn',cm.sum() - (TP + FN + FP)) # type: ignore
print('tn',cm.ravel()[0:3:2].sum()+cm.ravel()[6:9:2].sum()) # type: ignore

tn 151
tn 151


In [10]:
#create a multiprocess function to run the following code in parallel
from doctest import master
import re
import traceback

def process_each_ledger(key):
    ledger_df = pd.DataFrame()
    try:
        if key != 'name':  
            case_df = pd.DataFrame()
            if master_ledger[key]['aggregated_classification_reports'] is None:
                case_df['f1 binary'] = [0]
                pass
            else: 
                case_df['f1 binary']= master_ledger[key]['aggregated_classification_reports'][1]['f1-score'] # 2= stress for wesad, but for E2E, it's shifted to one via utilz.transform_labels
                pass
            case_df['is_done'] = master_ledger[key]['is_done']
            case_df['it'] = key.split('_')[-1]
            case_df['model'] = key.split('_')[-3]
            case_df['tune'] = key.split('_')[-4]
            case_df['fold'] = key.split('_')[-6]
            case_df['name'] = key
            
            cm = master_ledger[key]['aggregated_confusion_matrices']
            if cm is None:
                case_df['TP'] = 0
                case_df['FP'] = 0
                case_df['FN'] = 0
                case_df['TN'] = 0
                case_df['f1 binary'] = 0
                case_df['TPR'] = 0
                case_df['TNR'] = 0 
                case_df['PPV'] = 0
                case_df['NPV'] = 0
                case_df['FPR'] = 0
                case_df['FNR'] = 0
                case_df['FDR'] = 0
                case_df['specificity'] = 0
                ledger_df = pd.concat([ledger_df, master_ledger[key]['df_best_model'],
                                   master_ledger[key]['df_metrics'], case_df], axis = 1) # type: ignore
                ledger_df['status'] = 'error - cm none'
                
            elif cm.shape == (3,3):
                a = cm.ravel()[0] + cm.ravel()[2] + cm.ravel()[6] + cm.ravel()[8]
                b = cm.ravel()[1] + cm.ravel()[7]
                c = cm.ravel()[3] + cm.ravel()[5]
                d = cm.ravel()[4]
                tp = d
                tn = a
                fp = b
                fn = c
                perc = [tp/ (tp+fp) if (tp + fp) != 0 else 0]
                recall = [tp / (tp + fn) if (tp + fn) != 0 else 0]
                f1_bin = 2 * (perc[0] * recall[0]) / (perc[0] + recall[0]) if (perc[0] + recall[0]) != 0 else 0

                case_df['TP'] = cm.ravel()[4] # type: ignore
                case_df['FP'] = cm.ravel()[1:8:6].sum() # type: ignore
                case_df['FN'] = cm.ravel()[3:6:2].sum() # type: ignore
                case_df['TN'] = cm.sum() - (case_df['FP'] + case_df['FN'] + case_df['TP']) # type: ignore
                case_df['f1 binary'] = f1_bin

                # Sensitivity, hit rate, recall, or true positive rate
                case_df['TPR'] = case_df['TP']/(case_df['TP']+case_df['FN'])
                # Specificity or true negative rate
                case_df['TNR'] = case_df['TN']/(case_df['TN']+case_df['FP']) 
                # Precision or positive predictive value
                case_df['PPV'] = case_df['TP']/(case_df['TP']+case_df['FP'])
                # Negative predictive value
                case_df['NPV'] = case_df['TN']/(case_df['TN']+case_df['FN'])
                # Fall out or false positive rate
                case_df['FPR'] = case_df['FP']/(case_df['FP']+case_df['TN'])
                # False negative rate
                case_df['FNR'] = case_df['FN']/(case_df['TP']+case_df['FN'])
                # False discovery rate
                case_df['FDR'] = case_df['FP']/(case_df['TP']+case_df['FP'])
                #specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
                case_df['specificity'] = [((tn / (tn + fp)) if (tn + fp) != 0 else 0)]
                ledger_df = pd.concat([ledger_df, master_ledger[key]['df_best_model'],
                                   master_ledger[key]['df_metrics'], case_df], axis = 1) # type: ignore
                ledger_df['status'] = 'success'
            elif cm.shape == (2,2):
                a = cm.ravel()[0]
                b = cm.ravel()[1]
                c = cm.ravel()[2]
                d = cm.ravel()[3]
                tp = d
                tn = a
                fp = b
                fn = c
                perc = [tp/ (tp+fp) if (tp + fp) != 0 else 0]
                recall = [tp / (tp + fn) if (tp + fn) != 0 else 0]
                f1_bin = 2 * (perc[0] * recall[0]) / (perc[0] + recall[0]) if (perc[0] + recall[0]) != 0 else 0

                case_df['TP'] = tp
                case_df['FP'] = fp
                case_df['FN'] = fn
                case_df['TN'] = tn
                case_df['f1 binary'] = f1_bin

                # Sensitivity, hit rate, recall, or true positive rate
                case_df['TPR'] = case_df['TP']/(case_df['TP']+case_df['FN'])
                # Specificity or true negative rate
                case_df['TNR'] = case_df['TN']/(case_df['TN']+case_df['FP']) 
                # Precision or positive predictive value
                case_df['PPV'] = case_df['TP']/(case_df['TP']+case_df['FP'])
                # Negative predictive value
                case_df['NPV'] = case_df['TN']/(case_df['TN']+case_df['FN'])
                # Fall out or false positive rate
                case_df['FPR'] = case_df['FP']/(case_df['FP']+case_df['TN'])
                # False negative rate
                case_df['FNR'] = case_df['FN']/(case_df['TP']+case_df['FN'])
                # False discovery rate
                case_df['FDR'] = case_df['FP']/(case_df['TP']+case_df['FP'])
                #specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
                case_df['specificity'] = [((tn / (tn + fp)) if (tn + fp) != 0 else 0)]
            
                ledger_df = pd.concat([ledger_df, master_ledger[key]['df_best_model'],
                                   master_ledger[key]['df_metrics'], case_df], axis = 1) # type: ignore
                ledger_df['status'] = 'success'
            else:
                case_df['TP'] = 0
                case_df['FP'] = 0
                case_df['FN'] = 0
                case_df['TN'] = 0
                case_df['f1 binary'] = 0
                
                case_df['TPR'] = 0
                case_df['TNR'] = 0 
                case_df['PPV'] = 0
                case_df['NPV'] = 0
                case_df['FPR'] = 0
                
                case_df['FNR'] = 0
                case_df['FDR'] = 0
                case_df['specificity'] = 0
                
                ledger_df = pd.concat([ledger_df, master_ledger[key]['df_best_model'],
                                   master_ledger[key]['df_metrics'], case_df], axis = 1)
                ledger_df['status'] = f'error - {type(cm)} not supported'
    except Exception as e:
        # Log the exception
        logging.error(f"Error processing ledger {i}: {e}")
        logging.error(f'cm \n{cm}')
        logging.error(f'key {key}')
        # Also log the full exception traceback
        logging.error(traceback.format_exc())
        
    return ledger_df

In [17]:
import concurrent.futures
import logging

#logging.basicConfig(filename='logs/analyze_all_case_results.log',level=logging.INFO)
logging.basicConfig(filename='logs/analyze_all_case_results.log',filemode='w', level=logging.INFO)

if __name__ == '__main__':
    ledger_results = []
    force_update = False # range once on. 11/28/2023
    if os.path.isfile('/mnt/d/Users/alkurdi/ledger_results.pkl') and force_update != True:
        with open('/mnt/d/Users/alkurdi/ledger_results.pkl', 'rb') as handle:
            ledger_results = pickle.load(handle)
        print('loaded ledger_results')
    else:
        print('file not found or force_update is True')
        start_time = time.time()
        keychain = list(master_ledger.keys())
        master_ledger_df = pd.DataFrame()
        with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
                futures = [executor.submit(process_each_ledger, i) for i in keychain]
                done, not_done = concurrent.futures.wait(futures, return_when='ALL_COMPLETED')
                logging.debug(f'done assigning futures, now waiting for them to finish')
                for future in done:
                    ledger_results = future.result()
                    master_ledger_df = pd.concat([master_ledger_df, ledger_results], axis = 0)
        imax =len(master_ledger)
        logging.info(f'finished {imax} iterations in {round(time.time()-start_time,2)} seconds')
        logging.info(f'an average of {(time.time()-start_time)/imax} seconds per iteration')

        with open('/mnt/d/Users/alkurdi/master_ledger_df.pkl', 'wb') as f:
                    pickle.dump(master_ledger_df, f)

file not found or force_update is True


In [18]:
print(f'we have about {-len(master_ledger_df)+ len(master_ledger)} fucked up cases')

we have about 1 fucked up cases


In [41]:
n_not_done = (master_ledger_df['is_done']==False).sum()
n_done = (master_ledger_df['is_done']==True).sum()
n_neigther = ((master_ledger_df['is_done']!= True ) & (master_ledger_df['is_done']!= False) ).sum()
print(f' not done {n_not_done}. done {n_done}. else {n_neigther}. tot = {n_done+n_neigther+n_not_done}')

 not done 7419. done 4557. else 1088. tot = 13064


In [43]:
master_ledger_df[master_ledger_df['is_done']==True]

,f1 binary,is_done,it,model,tune,fold,name,TP,FP,FN,...,best_model_train_acc,best_model_val_acc,best_model_learning_rate,best_model_nb_epoch,precision,accuracy,recall,duration,f1,auc
0,0.822695,True,01,fcnM,02,02,delta old e2e_snr_0.2_snr_0.2_WESAD_5fold_02_t...,58,18,7,...,0.826284,0.793722,0.0,15.0,0.520357,0.631818,0.538618,135.227864,0.527129,0.778918
0,0.773810,True,02,fcnM,02,02,delta old e2e_snr_0.2_snr_0.2_WESAD_5fold_02_t...,65,38,0,...,0.903323,0.780269,0.0,41.0,0.578913,0.695455,0.612842,245.804731,0.573780,0.797865
0,0.019417,True,02,fcnM,01,01,delta old e2e_copy_snr_0.2_WESAD_5fold_01_tune...,1,37,64,...,0.525680,0.497758,0.0,3.0,0.458476,0.440909,0.344340,107.592084,0.350584,0.563824
0,0.000000,True,01,mlpM,07,03,new_delta_results/results_reduced_WESAD_5fold_...,0,0,66,...,0.517964,0.532110,0.0,0.0,0.176560,0.529680,0.333333,4.689368,0.230846,0.500000
0,0.000000,True,03,fcnM,01,01,delta old e2e_copy_snr_0.2_WESAD_5fold_01_tune...,0,38,65,...,0.530212,0.497758,0.0,3.0,0.462542,0.454545,0.362880,107.300939,0.370175,0.559015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.710000,True,02,resnetM,09,04,bbyn katerina snr_0.5_WESAD_5fold_04_tune_09_r...,71,58,0,...,0.773071,0.834862,0.0,88.0,0.430885,0.632743,0.536723,3927.501450,0.459922,0.824222
0,0.797468,True,00,resnetM,09,04,bbyn katerina snr_0.5_WESAD_5fold_04_tune_09_r...,63,24,8,...,0.706505,0.715596,0.0,22.0,0.466799,0.694690,0.561311,2071.252654,0.509662,0.749011
0,0.945946,True,04,resnetM,08,04,bbyn katerina snr_0.5_WESAD_5fold_04_tune_08_r...,70,7,1,...,0.996974,0.788991,0.0,19.0,0.631055,0.725664,0.643648,907.909626,0.636663,0.809554
0,0.916667,True,04,fcnM,03,01,delta old e2e_snr_0.5_WESAD_5fold_01_tune_03_f...,55,0,10,...,0.694864,0.713004,0.0,106.0,0.708634,0.754545,0.669820,646.662103,0.685107,0.752110
